<a href="https://colab.research.google.com/github/orangeduice/mlDrarn/blob/main/dataProcesser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/Resources/')

Mounted at /Resources/


In [ ]:
!pip install pydarn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 9.8 MB/s eta 0:00:00
  Created wheel for aacgmv2: filename=aacgmv2-2.6.3-cp39-cp39-linux_x86_64.whl size=1685192 sha256=5ef27f64bd74425e7849de79f20f00a4eb3deaebd9c3988361d64295154a228e
  Stored in directory: /root/.cache/pip/wheels/2c/d6/78/d9c0996ae17c8605a3eabbe2a61dc205a6a9493538bbecfb9a
Successfully built aacgmv2


In [ ]:
!pip install jdcal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
import pydarn
import bz2
import datetime
import numpy as np
import pandas as pd
import os
import jdcal

import time

YEAR = 1996


IMPORTANT: Please make sure to cite pyDARN in publications that use plots created by pyDARN using DOI: https://zenodo.org/record/3727269. Citing information for SuperDARN data is found at https://pydarn.readthedocs.io/en/master/user/citing/


In [ ]:
def extendedJD(year, month, day, hour, minute, second):
    JulianDate = sum(jdcal.gcal2jd(year, month, day))
    ExtendedPart =  hour/24 + minute/1440 + second/86400
    return JulianDate + ExtendedPart


def calculate_glt_lat(stationID,bmnum,slist,elv,fg):
    thing = pydarn.Coords.GEOGRAPHIC(stid = stationID, gates = (int(slist),int(slist)),frang = fg, elv_angle = elv)
    lat = thing[0][0][int(bmnum)]
    lon = thing[1][0][int(bmnum)] 
    return lat


def calculate_glt_lon(stationID,bmnum,slist,elv,fg):
    thing = pydarn.Coords.GEOGRAPHIC(stid = stationID, gates = (int(slist),int(slist)),frang = fg, elv_angle = elv)
    lat = thing[0][0][int(bmnum)]
    lon = thing[1][0][int(bmnum)] 
    return lon
    
    

def DataProcessing(fitacf_FilePath):
    #unzip bz2
    with bz2.open(fitacf_FilePath) as fp:
        fitacf_stream = fp.read()
        
    reader = pydarn.SuperDARNRead(fitacf_stream, True)
    ord_list = reader.read_fitacf()
    
    stationID = ord_list[0]['stid']
    
    #parameters to use in ML data
    
    df = pd.DataFrame(ord_list)
    
    
    df = df.rename(columns={"time.yr":"yr", "time.mo":"mo", "time.dy":"dy", "time.hr":"hr", "time.mt":"mt", "time.sc":"sc"})
    
    df["jd"] = df.apply(lambda x: extendedJD(x.yr, x.mo, x.dy, x.hr, x.mt, x.sc), axis=1)
    
    
    
    #df["time.jd"] = (df["time.yr"]*365 +  df["time.mo"]*30 + df["time.dy"] + df["time.hr"]/24 + df["time.mt"]/1440 + df["time.sc"]/86400)
    
    
    df = df.loc[:,["jd","slist","v","p_l","w_l","elv","gflg","bmnum","bmazm","frang","rsep"]]
    
    #df = df.loc[:,["slist","v","p_l","w_l","elv","gflg","bmnum","bmazm","frang","rsep"]]
    
    df = df.explode(["slist","v","p_l","w_l","elv","gflg"])
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(value = 0, inplace = True)
    #print(df)
    #df["Lat"] = df.apply(lambda x: calculate_glt_lat(stationID, x.bmnum, x.slist, x.elv, x.frang) , axis=1)
    #df["Lon"] = df.apply(lambda x: calculate_glt_lon(stationID, x.bmnum, x.slist, x.elv, x.frang) , axis=1)
    df = df.loc[:,["jd","gflg"]]
    
    return df

In [ ]:
StartingDir = "/Resources/MyDrive/Masters/un-processed/"
EndingDir = "/Resources/MyDrive/Masters/processed/1996/temp/"

 

data = [[],[],[],[],[],[],[],[],[],[],[],[]]

In [ ]:
fileNum = 0


for filename in os.listdir(StartingDir): #make function to generlize for any year
    
    f = os.path.join(StartingDir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        start = time.time()
        head, tail = os.path.split(f)


        #print("Processing",tail)

        month = int((tail.split('.')[0])[4:6])

        #print(month)

        ProcessedData_df = DataProcessing(f)

        data[month-1].append(ProcessedData_df)
        fileNum = fileNum + 1





        # fitacf_file = f
        # ProcessedData_df = DataProcessing(fitacf_file)
        # #Save the 2 hour period
        # SaveDataToCSV(ProcessedData_df, EndingDir + "\\" + os.path.basename(f)[:-10]+"processedRCC.csv")
        end = time.time()
        print("done: ", tail)
        print("Time taken: ", end-start)


done:  19960119.1200.00.han.fitacf.bz2
Time taken:  3.261274576187134
done:  19960118.1600.00.han.fitacf.bz2
Time taken:  2.0144832134246826
done:  19960119.2200.00.han.fitacf.bz2
Time taken:  3.372875690460205
done:  19960119.0000.00.han.fitacf.bz2
Time taken:  1.8279876708984375
done:  19960118.0400.00.han.fitacf.bz2
Time taken:  1.1667215824127197
done:  19960119.0800.00.han.fitacf.bz2
Time taken:  2.940781354904175
done:  19960118.0800.00.han.fitacf.bz2
Time taken:  3.311349391937256
done:  19960118.0200.00.han.fitacf.bz2
Time taken:  1.0073246955871582
done:  19960118.1400.00.han.fitacf.bz2
Time taken:  2.1298866271972656
done:  19960119.1600.00.han.fitacf.bz2
Time taken:  1.8886241912841797
done:  19960118.0600.00.han.fitacf.bz2
Time taken:  2.7546451091766357
done:  19960118.1200.00.han.fitacf.bz2
Time taken:  2.892472505569458
done:  19960118.1000.00.han.fitacf.bz2
Time taken:  3.656294345855713
done:  19960118.0000.00.han.fitacf.bz2
Time taken:  2.005491256713867
done:  199601

In [ ]:
MONTH = 1
for mo in data:
    if len(mo) != 0:
        df = pd.concat(mo)
        df.to_csv(EndingDir + "/" +str(YEAR)+str(MONTH)+"processedRCC.csv")
    MONTH = MONTH + 1